TWITTER SENTIMENT ANALYSIS

In [ ]:
#import necessary modules
import pyforest

In [ ]:
#read in the data
df = pd.read_csv(r"C:\Users\User\Documents\training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", header=None )
df = df.head(1000)

In [ ]:
#rename columns
df.columns = ['sentiment_value','id','date','query','user','tweet']

In [ ]:
#display data
df.head()

In [ ]:
df['tweet'].values[0]


In [ ]:
df.shape

In [ ]:
df.info

BASIC EDA 

In [ ]:
df['sentiment_value'].value_counts().sort_index()\
     .plot(kind='bar', title='Sentiment Value Counts')
plt.show()

NLTK PROCESSES

In [ ]:
#display tweet
example = df['tweet'][50]
print(example)

In [ ]:
#tokenize tweet
tokens = nltk.word_tokenize(example)

In [ ]:
#part of speech tagging
tagged = nltk.pos_tag(tokens)
tagged[:]

In [ ]:
#groups tokens
entities = nltk.chunk.ne_chunk(tagged)
entities.pprint()

VADER SENTIMENT SCORING

We use VADER from NLTK to get the negative, neutral and positive sentiments of the tweets
- stop words (eg 'and', 'the') are removed 
- each word is scored and total scores combined
- does not account of relationships between words

In [ ]:
import tqdm
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [ ]:
#test sia on a text
sia.polarity_scores('I am happy!')

In [ ]:
#test sia on a text
sia.polarity_scores('I am sad!')

In [ ]:
#apply on tweet
sia.polarity_scores(example)

In [ ]:
#apply sia on all tweets
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['tweet']
    myid = row['id']
    res[myid] = sia.polarity_scores(text)

In [ ]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
vaders = vaders.merge(df, how = 'left')

In [ ]:
#sentiment score and metadata
vaders.head()

In [ ]:
#plout vader results
ax = sns.barplot(data=vaders, x='sentiment_value', y='compound')
ax.set_title("Compound Score by Tweet Sentiment")
plt.show()

In [ ]:
sns.barplot(data=vaders, x='sentiment_value', y='neg')
plt.show()

ROBERTA PRE-TRAINED MODEL

-Model trained on large data

-accounts for words and and context too

In [ ]:
#install requirements
%pip install transformers
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
#install model
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
print(example)

In [ ]:
#test roberta model
encoded = tokenizer(example, return_tensors = 'pt')
output = model(**encoded)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'negative': scores[0],
    'neutral': scores[1],
    'positive' : scores[2]
}
print(scores_dict)

In [ ]:
#iterate over tweets
def polarity_scores_roberta(example):
    encoded = tokenizer(example, return_tensors = 'pt')
    output = model(**encoded)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative': scores[0],
        'neutral': scores[1],
        'positive' : scores[2]
    }
    return scores_dict
print(scores_dict)

In [ ]:
#apply roberta on all tweets 
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['tweet']
    myid = row['id']
    vader_results = sia.polarity_scores(text)
    vader_results_rename = {}
    for key, value in vader_results.items():
        vader_results_rename[f'vader_{key}'] = value
    roberta_results = polarity_scores_roberta(text)
    both = {**vader_results, **roberta_results}
    res[myid] = both
    break

In [ ]:
result_df = pd.DataFrame(res).T 
result_df = result_df.reset_index().rename(columns={'index': 'id'})
result_df = result_df.merge(df, how = 'left')

In [ ]:
result.head()

NAIVE BAYES CLASSIFIER SENTIMENT ANALYSIS WITH ML MODEL

In [ ]:
df_two = pd.read_csv(r"C:\Users\User\Documents\training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", header=None )

In [ ]:
df_two.columns = ['sentiment_value','id','date','query','user','tweet']

In [75]:
#regex
import re

def preprocess(tweet):
    tweet = re.sub(r"http\S+", "", tweet)
    # Remove mentions
    tweet = re.sub(r"@\w+", "", tweet)
    # Remove special characters
    tweet = re.sub(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ", tweet)
    # Lowercase the text
    tweet = tweet.lower()
    return tweet

# Drop unnecessary columns
df_two = df[["sentiment_value", "tweet"]]

# Replace target values with labels
df_two["tweet"] = df["tweet"].replace({0: "negative", 4: "positive"})

# Apply preprocess to the text column
df_two["tweet"] = df["tweet"].apply(preprocess)


In [ ]:
df_two.head()

In [ ]:
#split data and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["tweet"], df["sentiment_value"], test_size=0.2, random_state=42)

In [ ]:
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)


In [ ]:
#apply logistic regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_features, y_train)

In [ ]:
accuracy = model.score(X_test_features, y_test)
print("Accuracy:", accuracy)
